# Prep script

In [1]:
import time
import pylast
import hashlib
from datetime import date
from datetime import datetime
import requests
import pandas as pd, requests, time, json, os.path
import logging as lg, datetime as dt
import pandas as pd, numpy as np, matplotlib.pyplot as plt, logging as lg
import time, requests, re, json, os.path, datetime as dt
from geopy.distance import great_circle
import mpl_toolkits
import sys
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
%run all_recently_played_music.py
%run get_locations.py
%run geocode_locations.py

<br>

# Set-up API call information

### Last.FM

In [3]:
# Enter your user details
user_name = 'pollystewart'
user_password = '77ebJ!134'
api_key = '040075c038c9f0e9200581b60723f860'
api_secret = '994d7d88b34019cb9c337fe179ac0f67'

# set-up hash / API call
password_hash = pylast.md5(user_password)
network = pylast.LastFMNetwork(api_key = api_key, api_secret =  api_secret, username = user_name, password_hash = password_hash)
user = pylast.User(user_name, network)

### Spotify

In [4]:
cid ="3fc91dc129ff4936beb0411d23003200" 
secret = "baa0c78d715f4497a8baedd3677516db"
sp = spotipy.Spotify()

<br>

# Final things to do

 - Create a folder where this notebook is saved called 'raw_data' to save checkpoints
 - Create a folder where this notebook is saved called 'geocode' to save all geocode-related caches/jsons
 - Create a folder where this notebook is saved called 'place_Data' to save all geocode-related data files

<br>

# Get all music played

In [8]:
# just gets latest 5 as a check
recent_tracks = user.get_recent_tracks(limit = 5, cacheable=True)

In [9]:
tracks = get_all_recent_tracks(user = user, network = network, limit_value = None)
tracks.head()

,artist,track,album,timestamp,track_duration,listener_count,playcount
0,Catfish and the Bottlemen,Fluctuate,Fluctuate,2019-11-20 11:34:23,0,29000,172060
1,Adele,Rumour Has It,21,2019-11-20 11:30:35,223000,646867,5038503
2,Yeah Yeah Yeahs,Heads Will Roll (A-Trak Remix),Heads Will Roll (A-trak Remix),2019-11-20 11:24:11,383000,217777,1280431
3,Ke$ha,Die Young,Warrior (Deluxe Version),2019-11-20 11:20:54,213000,408202,4871835


In [10]:
tracks_w_tags = get_all_tags(tracks, network)

In [8]:
#tracks_w_tags.to_csv('raw_data/music_with_tags_20193110.csv', index = False)

<br>

# Get MBID of all artists/tracks/albums

In [9]:
df_mbid = get_mbid_of_all_rows(tracks_w_tags, network)

In [10]:
#df_mbid.to_csv('raw_data/music_with_mbid_20193110.csv', index = False)

<br>

# Get Locations & MusicBrainz Info

In [11]:
all_data = find_all_locations(df_mbid)

there are 187 unique artists to get details for
processed 187 artists in 83.8 seconds and saved csv
0 artists to retry
processed 0 artists in 0.0 seconds and saved csv
we will attempt to get place full names for 112 place IDs
finished getting place full names from place IDs in 77.33 seconds


/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [12]:
#all_data.to_csv("raw_data/music_with_places_20193110.csv", index = False)

<br>

# Get lat/lon coordinates

In [13]:
music_with_lat_lon = get_all_latlons_for_music(all_data, ignore_country_if_more_detail = False)

437 total artists
111 unique places
geocoded 111 addresses in 221.00 seconds
received 108 non-null lat-longs
3 addresses still lack lat-long
geocoded 3 addresses in 0.00 seconds
received 108 non-null lat-longs


In [14]:
#music_with_lat_lon.to_csv("raw_data/music_with_places_latlon_20193110.csv", index = False)

<br>

# Get info from Spotify

In [15]:
spotify_data = get_spotipy_id(music_with_lat_lon, sp = sp, cid = cid, secret = secret)
spotify_data = spotify_data.drop_duplicates(["artist", "track"])

retrying ...1secs


In [16]:
spotify_data_info = get_spotify_data(spotify_data, batchsize_value = 100,  cid = cid, secret = secret)

In [17]:
spotify_info = pd.merge(spotify_data, spotify_data_info, on = "track_id", how = "left", validate = "one_to_one" )

In [18]:
spotify_info = spotify_info[['artist', 'track', 'popularity', 'acousticness', 'danceability', 'duration_ms', 
                             'energy','instrumentalness', 'liveness','loudness','mode','speechiness','tempo',
                             'time_signature', 'valence']]

In [19]:
music_with_all_features = pd.merge(music_with_lat_lon, spotify_info, on = ["artist", "track"], how = "left", validate = "many_to_one" )


In [20]:
#music_with_all_features.to_csv("raw_data/music_with_all_features_20193110.csv", index = False)